---
format:
  html:
    code-line-numbers: true
    code-overflow: wrap
    code-block-bg: true
    code-block-border-left: true
    highlight-style: Arrow
---

# The Knapsack Problem {#sec-knapsack}

The knapsack problem is a classic optimization problem in the field of operations research. 
It involves selecting a subset of items from a given set of items to maximize the total value/profit while satisfying certain constraints.
This problem is NP-hard, meaning that it is computationally difficult to find an optimal solution for large instances of the problem. Various algorithms have been developed to solve this problem, including dynamic programming, branch and bound, and heuristic methods. The knapsack problem has applications in a variety of fields, including computer science, finance, and logistics, among others.
In this chapter, we'll examine two variants of the knapsack problem, namely, the single-dimensional knapsack problem and the multi-dimensional knapsack problem.

## Single-dimensional Knapsack Problem

In the single-dimensional knapsack problem (SDKP), we are given a set of $n$ items, each with a weight $w_i$ and a value $v_i$, and a knapsack with a maximum weight capacity $W$, the goal is to select a subset of items such that the sum of their weights is less than or equal to $W$, and the sum of their values is maximized.
The SDKP can be formally formulated as follows (@dudzinski_exact_1987).

\begin{align}
    \text{max.} &\quad \sum_{i = 1}^nv_i x_i \label{knapsack1-obj} \\
    \text{s.t.} &\quad \sum_{i=1}^n w_i x_i \leq W \label{knapsack1-cons1} \\
    &\quad x_i \in \{0, 1\}, \ i = 1, \cdots, n \label{knapsack1-cons2}
\end{align}

In the model, the $n$ represents the total amount of items that are being considered. 
The binary decision variable $x_i$ has a value of 1 when item $i$ is chosen, and 0 when it's not. The value or profit of selecting item $i$ is indicated by $v_i$, and the weight of item $i$ is represented by $w_i$. Finally, $W$ specifies the maximum weight capacity of the knapsack.

To represent the list of items in the problem, we first define a class `Item` that has three attributes:

- `_index`: this is the index of an item and starts from 0
- `_profit`: this is the profit or value of selecting the item
- `_properties`: this dictionary saves an item's properties, including weight

In [82]:
class Item:
    """An item represents an object that can be placed within a knapsack
    """
    
    def __init__(self, index, profit):
        """constructor

        Args:
            index (int): index of the item, starting from 0
            profit (float): profit of choosing the item
        """
        self._index = index
        self._profit = profit
        self._properties = {}
    
    @property
    def index(self): return self._index
    
    @property
    def profit(self): return self._profit
    
    def get_property(self, name):
        return self._properties[name]

    def set_property(self, name, value):
        self._properties[name] = value
        
    def __str__(self):
        p_str = ""
        for attr in self._properties:
            p_str += f'{attr}: {self._properties[attr]}'
        return f"index: {self._index}, profit: {self._profit}, " + \
                p_str

Next, we define a class `KnapsackDataCenter` to hold all the information we need to solve a knapsack problem.
The class has two attributes:

- `_items`: this is the full list of items being considered for putting into the knapsack
- `_capacities`: this saves the maximal capacity of the knapsack corresponding to different properties, including weight.

The class also defines a method `read_data_set_f()` that reads and parses some benchmarking instancese available online.

In [79]:
class KnapsackDataCenter:
    
    def __init__(self):
        self._items = []
        self._capacities = {}
        
    def read_data_set_f(self, data_file: str):
        """this function reads and parses data presented in 
        http://artemisa.unicauca.edu.co/~johnyortega/instances_01_KP/

        Args:
            data_file (str): path to the data file
            num_items (int): number of items in the file
            capacity (int): knapsack capacity
        """
        with open(data_file) as f:
            first_line = f.readline()
            num_items, capacity = first_line.split()
            self._capacities['weight'] = float(capacity)
            
            item_idx = 0
            rest_lines = f.readlines()
            for line in rest_lines:
                profit, weight = line.split()
                item = Item(item_idx, float(profit))
                item.set_property('weight', float(weight))
                self._items.append(item)
                item_idx += 1
                if item_idx == int(num_items): break
                
    @property
    def items(self): return self._items
    
    @property
    def capacities(self): return self._capacities

The code snippet below reads the instance `f1_l-d_kp_10_269` and shows its key information.

In [84]:
data_file = "./data/knapsack/instances_01_KP/low-dimensional/f1_l-d_kp_10_269"

data_center = KnapsackDataCenter()
data_center.read_data_set_f(data_file)

capacities = data_center.capacities
print(f"Knapsack info: {capacities}")

items = data_center.items
print("Item info: ")
for item in items:
    print(item)

Knapsack info: {'weight': 269.0}
Item info: 
index: 0, profit: 55.0, weight: 95.0
index: 1, profit: 10.0, weight: 4.0
index: 2, profit: 47.0, weight: 60.0
index: 3, profit: 5.0, weight: 32.0
index: 4, profit: 4.0, weight: 23.0
index: 5, profit: 50.0, weight: 72.0
index: 6, profit: 8.0, weight: 80.0
index: 7, profit: 61.0, weight: 62.0
index: 8, profit: 85.0, weight: 65.0
index: 9, profit: 87.0, weight: 46.0


Now we are ready to solve the SDKP using OR-Tools.
The code below shows the completion definition of the class `SDKnapsackSolver` which has a number of attributes:

- `_data_center`: this should be an instantiated `KnapsackDataCenter` object
- `_solver`: this is the solver object used for modeling and problem solving
- `_var_x`: this is the object that holds all the decision variables used in the model
- `_opt_obj`: this is the optimal solution value found by the solver
- `_opt_x`: this is the optimal solution identified when the solving process completes

The class has two major methods:

- `build_model()`: this is responsible for instantiating the solver object, creating variables, generating constraints and defining the objective function
- `optimize()`: this is the place where the OR-Tools searches for the optimal solution

In [87]:
from ortools.linear_solver import pywraplp

class SDKnapsackSolver:
    
    def __init__(self, data_center):
        self._data_center = data_center

        self._solver = None
        self._var_x = None

        self._opt_obj = None
        self._opt_x = None
        
    def build_model(self):
        self._solver = pywraplp.Solver.CreateSolver('SCIP')

        self._create_variables()
        self._create_objective()
        self._create_constraints()
        
    def optimize(self):
        status = self._solver.Solve()
        if status == pywraplp.Solver.OPTIMAL:
            self._opt_obj = self._solver.Objective().Value()
            items = self._data_center.items
            self._opt_x = [
                    self._var_x[item.index].solution_value()
                    for item in items
                ]
    
    def _create_variables(self):
        items = self._data_center.items
        self._var_x = [self._solver.BoolVar(name=f'x_{i}')
                    for i, item in enumerate(items)]
    
    def _create_objective(self):
        items = self._data_center.items
        obj_expr = [
                self._var_x[item.index] * item.profit
                for item in items
                ]
        self._solver.Maximize(self._solver.Sum(obj_expr))
        
    def _create_constraints(self):
        items = self._data_center.items
        capacities = self._data_center.capacities
        expr = [
            self._var_x[item.index] * 
                item.get_property('weight')
            for item in items
            ]
        self._solver.Add(
                self._solver.Sum(expr) <= capacities['weight']
            )
        
    @property
    def opt_obj(self): return self._opt_obj
    
    def get_num_chosen_items(self):
        return sum(self._opt_x)

We employ the model to tackle 10 benchmark instances that were downloaded from an online source (http://artemisa.unicauca.edu.co/~johnyortega/instances_01_KP/). 
@tbl-knapsack-data displays the computational results.

More specifically, the first column of the table lists the names of the instances, while the second and third columns indicate the number of items and the capacity of the knapsack for each instance, respectively. The last two columns report the optimal solution's objective value and the number of selected items, respectively.

In [98]:
# | echo: false
# | label: tbl-knapsack-data
# | tbl-cap: Computational results of Knapsack problems

from IPython.display import Markdown
from tabulate import tabulate
import os
import re

# helper function to perform sort
def num_sort(test_string):
    return list(map(int, re.findall(r'\d+', test_string)))[0]

data_dir = "./data/knapsack/instances_01_KP/low-dimensional/"
dir_list = os.listdir(data_dir)
dir_list.sort(key=num_sort)

table_data = []
for file in dir_list:
    data_file = os.path.join(data_dir, file)

    data_center = KnapsackDataCenter()
    data_center.read_data_set_f(data_file)

    knapsack_solver = SDKnapsackSolver(data_center)
    knapsack_solver.build_model()
    knapsack_solver.optimize()
    
    row_data = []
    row_data.append(file)
    row_data.append(len(data_center.items))
    row_data.append(data_center.capacities['weight'])
    row_data.append(knapsack_solver.opt_obj)
    row_data.append(knapsack_solver.get_num_chosen_items())
    table_data.append(row_data)

col_names = ["Instance", "No. of Items", "Capacity", "Optimal Value", "No. of Chosen Items"]
Markdown(tabulate(table_data, headers=col_names))

Instance              No. of Items    Capacity    Optimal Value    No. of Chosen Items
------------------  --------------  ----------  ---------------  ---------------------
f1_l-d_kp_10_269                10         269          295                          6
f2_l-d_kp_20_878                20         878         1024                         17
f3_l-d_kp_4_20                   4          20           35                          3
f4_l-d_kp_4_11                   4          11           23                          2
f5_l-d_kp_15_375                15         375          481.069                      9
f6_l-d_kp_10_60                 10          60           52                          7
f7_l-d_kp_7_50                   7          50          107                          2
f8_l-d_kp_23_10000              23       10000         9767                         11
f9_l-d_kp_5_80                   5          80          130                          4
f10_l-d_kp_20_879               20         879         1025                         17

## Multi-dimensional Knapsack Problem